In [71]:
!pip install pyqt5 -q
!pip install pyqtwebengine -q
!pip install navec solvent -q
from navec import Navec
import pandas as pd
import torch
import numpy as np
#from slovnet.model.emb import NavecEmbedding

In [7]:
#!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2024-02-20 20:43:33--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Распознаётся storage.yandexcloud.net (storage.yandexcloud.net)… 213.180.193.243, 2a02:6b8::1d9
Подключение к storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 53012480 (51M) [application/x-tar]
Сохранение в: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50,56M  10,7MB/s    за 4,7s    

2024-02-20 20:43:38 (10,7 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ сохранён [53012480/53012480]



In [39]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [199]:
path = '/home/daniil/Рабочий стол/CLASSIFICATION PROBLEM/JobMatchRecommenderSystem/Scripts/Ready_to_use_data'
df = pd.read_csv(path)
df = df.iloc[np.where((df.department != ' ') & (df.unit != ' '))]
df.full_description = df.department + ' ' + df.unit + ' ' + df.job_name

In [200]:
df = df[['func_1','full_description']]

In [201]:
df.head()

,func_1,full_description
0,Маркетинг,маркетинг дизайн и онлайн продажи департамент ...
1,Инвестиционная деятельность,управление крупным частным капиталом департаме...
2,Корпоративный бизнес,крупный корпоративный и инвестиционный бизнес ...
3,Управление рисками,управление рисками департамент рисков рознично...
4,Корпоративный бизнес,малый и микро корпоративный бизнес департамент...


In [202]:
#Проверим, какой процент слов из нашего датасета не знаком библиотеке Navec.


unique_words = set(" ".join(df.full_description).split(' '))

cnt = 0
for word in unique_words:
    if word not in navec:
        cnt+=1
print( (cnt/len(unique_words))*100 )   

20.79141515761234


In [203]:
# Уберём все неизвестные для Navec слова из описаний объектов из запишем это в колонку ready_to_embed.

def embedding_condition(sent):
    sent = sent.split(' ')
    for word in sent:
        if word not in navec:
            sent.remove(word)
    " ".join(sent)       
    return sent

good_words = list(map(lambda x: embedding_condition(x), df.full_description.to_numpy()))

df['ready_to_embed'] = pd.Series(good_words)

In [204]:
def sent2vec(sent, embedder):
    sent_embedding = np.zeros(embedder.pq.dim)
    for word in sent:
        if word in embedder:
            sent_embedding += embedder[word]
    return sent_embedding

embeddings = list(map(lambda x: sent2vec(x, navec), df.ready_to_embed ) )

df['embeddings'] = embeddings

In [205]:
# Создаём словарь с нумерацией функций 1 уровня
func_1_classes = {}
for num, function in enumerate( df.func_1.unique() ):
    func_1_classes[function] = num

In [206]:
embeddings_df = pd.DataFrame(df['embeddings'].tolist()).to_numpy()

In [207]:
labels_df = df.func_1.to_numpy()

In [208]:
#df.to_csv('Ready_to_use_data_embeddings')

In [209]:
print(embeddings_df.shape)
print(labels_df.shape)

(11211, 300)
(11211,)


In [210]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [211]:
# В качестве признаков выступает векторизованное описание, в качестве лейблов выступают функции 1 уровня.
X_train, X_test, y_train, y_test = train_test_split(embeddings_df, labels_df, test_size=0.2,\
                                                    random_state=0, shuffle=True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8968, 300), (2243, 300), (8968,), (2243,))

##### LogReg

In [182]:
model = LogisticRegression(penalty='l2', n_jobs=-1, solver='lbfgs')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность Логистической регрессии:{acc}')

Точность Логистической регрессии:0.734284440481498


/home/daniil/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### SGDClassifier

In [187]:
model = SGDClassifier(loss='hinge', alpha=0.0001, max_iter=500, n_jobs=-1)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность SGDClassifier:{acc}')

Точность SGDClassifier:0.6937137761925992


##### LinearSVC

In [188]:
model = svm.LinearSVC(loss='squared_hinge', multi_class='ovr')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность LinearSVC:{acc}')

Точность LinearSVC:0.7547926883637985


/home/daniil/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


##### SVC

In [189]:
model = svm.SVC( kernel='rbf')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)

print(f'Точность SVC c rbf ядром:{acc}')

Точность SVC c rbf ядром:0.683459652251449
